# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [26]:
# import libraries
import pickle
import string
import unittest

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score, make_scorer
from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package punkt to C:\Users\rebecca-
[nltk_data]     barnes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\rebecca-
[nltk_data]     barnes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\rebecca-
[nltk_data]     barnes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rebecca-barnes\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///data/disaster_messages.db')
df = pd.read_sql_table('messages', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df.message
Y = df.loc[:, 'related':'direct_report']

### 2. Write a tokenization function to process your text data

In [4]:
print(X[0])

Weather update - a cold front from Cuba that could pass over Haiti


In [4]:
def tokenize(text):
    table = text.maketrans(dict.fromkeys(string.punctuation))
    words = word_tokenize(text.lower().strip().translate(table))
    words = [word for word in words if word not in stopwords.words('english')]
    lemmed = [WordNetLemmatizer().lemmatize(word) for word in words]
    lemmed = [WordNetLemmatizer().lemmatize(word, pos='v') for word in lemmed]
    stemmed = [PorterStemmer().stem(word) for word in lemmed]
    
    return stemmed

In [32]:
tokenize(X[0])

['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pa', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [23]:
pipeline = Pipeline([
    ('text', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42, verbose=5)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, random_state=42)
print('Split complete')
pipeline.fit(train_X, train_Y)

Split complete


C:\Users\rebecca-barnes\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 10


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


building tree 2 of 10
building tree 3 of 10


...

building tree 4 of 10
building tree 5 of 10


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.6s remaining:    0.0s


building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.6s finished


Pipeline(memory=None,
     steps=[('text', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...           oob_score=False, random_state=42, verbose=5, warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [34]:
y_preds = pipeline.predict(test_X)

In [35]:
results_dict = {}

for pred, label, col in zip(y_preds.transpose(), test_Y.values.transpose(), test_Y.columns):
    print(col)
    print(classification_report(label, pred))
    results_dict[col] = classification_report(label, pred, output_dict=True)

related
              precision    recall  f1-score   support

           0       0.61      0.44      0.51      1563
           1       0.84      0.91      0.87      4991

   micro avg       0.80      0.80      0.80      6554
   macro avg       0.73      0.68      0.69      6554
weighted avg       0.78      0.80      0.79      6554

request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5443
           1       0.83      0.41      0.55      1111

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.86      0.70      0.74      6554
weighted avg       0.88      0.89      0.87      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      655

C:\Users\rebecca-barnes\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6498
           1       0.00      0.00      0.00        56

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.98      0.99      0.99      6554

shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6530
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6473
           1       0.00      0.00      0.00        81

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98   

There are a number of categories with small numbers of messages where none were picked up:
- aid_centers
- shops
- hospitals
- tools
- offer

What is the overall performance of the model?

In [36]:
weighted_avg = {}
for key in results_dict.keys():
    weighted_avg[key] = results_dict[key]['weighted avg']

df_wavg = pd.DataFrame(weighted_avg).transpose()
df_wavg

,f1-score,precision,recall,support
related,0.787986,0.784821,0.799817,6554.0
request,0.870000,0.881224,0.886329,6554.0
offer,0.992454,0.989955,0.994965,6554.0
aid_related,0.745885,0.750029,0.751297,6554.0
medical_help,0.895048,0.901214,0.921727,6554.0
medical_products,0.928948,0.936818,0.949344,6554.0
search_and_rescue,0.965270,0.966259,0.975893,6554.0
security,0.973671,0.969479,0.981996,6554.0
military,0.958667,0.956675,0.969179,6554.0
child_alone,1.000000,1.000000,1.000000,6554.0


In [37]:
df_wavg['f1-score'].describe()

count    36.000000
mean      0.932243
std       0.062012
min       0.745885
25%       0.919750
50%       0.941502
75%       0.976655
max       1.000000
Name: f1-score, dtype: float64

What are the lowest f1-scores?

In [38]:
df_wavg[df_wavg['f1-score'] <= df_wavg['f1-score'].quantile(0.25)]

,f1-score,precision,recall,support
related,0.787986,0.784821,0.799817,6554.0
request,0.870000,0.881224,0.886329,6554.0
aid_related,0.745885,0.750029,0.751297,6554.0
medical_help,0.895048,0.901214,0.921727,6554.0
shelter,0.919704,0.925902,0.933018,6554.0
other_aid,0.816513,0.821371,0.867867,6554.0
infrastructure_related,0.907733,0.898265,0.936527,6554.0
weather_related,0.854415,0.858867,0.861611,6554.0
direct_report,0.811850,0.824204,0.839945,6554.0


Let's collect these for future use.

In [39]:
key_categories = (df_wavg[df_wavg['f1-score'] <= df_wavg['f1-score'].quantile(0.25)].index).tolist()
key_categories

['related',
 'request',
 'aid_related',
 'medical_help',
 'shelter',
 'other_aid',
 'infrastructure_related',
 'weather_related',
 'direct_report']

What are the highest f1-scores?

In [40]:
df_wavg[df_wavg['f1-score'] >= df_wavg['f1-score'].quantile(0.75)]

,f1-score,precision,recall,support
offer,0.992454,0.989955,0.994965,6554.0
child_alone,1.000000,1.000000,1.000000,6554.0
clothing,0.984411,0.983971,0.986878,6554.0
missing_people,0.983324,0.977848,0.988862,6554.0
tools,0.990169,0.986921,0.993439,6554.0
hospitals,0.987202,0.982984,0.991456,6554.0
shops,0.994511,0.992690,0.996338,6554.0
aid_centers,0.981500,0.975435,0.987641,6554.0
fire,0.985135,0.983464,0.989777,6554.0


Some of the highest performing categories appear to be doing so because of imbalance - there are very few 1s in these categories and so their metrics are still high even if they are all classified as 0s. 

As a result, it's important to not be too influenced by these values. It seems reasonable to seek to improve the performance of the original "lowest" performing categories. 

In [41]:
df_wavg.loc[key_categories]['f1-score'].describe()

count    9.000000
mean     0.845460
std      0.058861
min      0.745885
25%      0.811850
50%      0.854415
75%      0.895048
max      0.919704
Name: f1-score, dtype: float64

The average f1-score for the key paramaters with the base model is 0.8436 with a std of 0.0597.

Let's create a function to easily assess a model's performance.

In [8]:
def model_performance(labels, preds):
    # Print/collect results
    results_dict = {}
    for pred, label, col in zip(preds.transpose(), labels.values.transpose(), labels.columns):
        print(col)
        print(classification_report(label, pred))
        results_dict[col] = classification_report(label, pred, output_dict=True)
    
    # Convert to df
    weighted_avg = {}
    for key in results_dict.keys():
        weighted_avg[key] = results_dict[key]['weighted avg']

    df_wavg = pd.DataFrame(weighted_avg).transpose()
    
    # Show metrics for "key categories"
    key_categories = ['related', 'request', 'aid_related', 'medical_help',
                      'shelter', 'other_aid', 'infrastructure_related',
                      'weather_related', 'direct_report']
    
    print(df_wavg.loc[key_categories]['f1-score'].describe())
    
    return df_wavg

### 6. Improve your model
Use grid search to find better parameters. 

In [42]:
pipeline.get_params()

{'memory': None,
 'steps': [('text',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x00000288FAA8E378>, use_idf=True,
           vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
               oob_score=False, random_state=42, 

Because of the imbalance in the classes, it is possible to get high accuracy scores even with all positive messages in a class falsely labeled as negative. As a result, I would like to use the f1-score when evaluating my models, with the micro average used, based on this [StackExchange answer](https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin). 

In [98]:
make_scorer(f1_score, average='micro')

parameters = {
    'text__ngram_range': ((1, 1), (1, 2)),
    'text__max_df': [0.5],
    'text__max_features': [5000],
    'text__use_idf': [False],
    'clf__estimator__max_depth': (25, 50, 100, None),
    'clf__estimator__min_samples_split': (2, 10, 25, 50, 100), 
    'clf__estimator__n_estimators': [200]
}

cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [93]:
cv.fit(train_X, train_Y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
b

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
b

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


...

building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200
building tree 44 of 200
building tree 45 of 200
building tree 46 of 200
building tree 47 of 200
building tree 48 of 200
building tree 49 of 200
building tree 50 of 200
building tree 51

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200
building tree 44 of 200
building tree 45 of 20

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    2.3s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('text', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...           oob_score=False, random_state=42, verbose=5, warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=3,
       param_grid={'text__ngram_range': ((1, 1), (1, 2)), 'text__max_df': [0.5], 'text__max_features': [5000], 'text__use_idf': [False], 'clf__estimator__max_depth': (2, 5, 10, 25), 'clf__estimator__n_estimators': (50, 100, 200)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [94]:
cv.best_params_

{'clf__estimator__max_depth': 25,
 'clf__estimator__n_estimators': 200,
 'text__max_df': 0.5,
 'text__max_features': 5000,
 'text__ngram_range': (1, 2),
 'text__use_idf': False}

In [95]:
cv.best_score_

0.21371172820669312

In [96]:
y_preds = cv.predict(test_X)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

In [97]:
results_cv = model_performance(test_Y, y_preds)

related
              precision    recall  f1-score   support

           0       0.80      0.07      0.14      1563
           1       0.77      0.99      0.87      4991

   micro avg       0.77      0.77      0.77      6554
   macro avg       0.79      0.53      0.50      6554
weighted avg       0.78      0.77      0.70      6554

request
              precision    recall  f1-score   support

           0       0.87      0.99      0.93      5443
           1       0.91      0.29      0.44      1111

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.89      0.64      0.69      6554
weighted avg       0.88      0.88      0.85      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      655

C:\Users\rebecca-barnes\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6456
           1       0.83      0.05      0.10        98

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.91      0.53      0.54      6554
weighted avg       0.98      0.99      0.98      6554

money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6421
           1       0.00      0.00      0.00       133

   micro avg       0.98      0.98      0.98      6554
   macro avg       0.49      0.50      0.49      6554
weighted avg       0.96      0.98      0.97      6554

missing_people
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6481
           1       0.00      0.00      0.00        73

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98 

These new parameters resulted in a reduction in performance.

### 8. Additional features:
Features that are common in improving nlp classification include ([Reference](https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/)):
- Number of words in sentence
- Number of characters in message
- Parts of speech count

These additional features can be added to the pipeline.

#### Number of words

In [9]:
class WordCount(BaseEstimator, TransformerMixin):
    def word_count(self, text):
        table = text.maketrans(dict.fromkeys(string.punctuation))
        words = word_tokenize(text.lower().strip().translate(table))
        return len(words)
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        count = pd.Series(x).apply(self.word_count)
        return pd.DataFrame(count)

In [10]:
class TestWordCountClass(unittest.TestCase):
    def setUp(self):
        self.sample_1 = WordCount().fit_transform('I went to the shops today.')
        self.sample_2 = WordCount().fit_transform("They've been very good to me, you know?")
        self.sample_3 = WordCount().fit_transform("In they went. Two by two.")
        self.sample_4 = WordCount().fit_transform("It takes a lot to love someone. \
            You would think that loving someone for 20 years would count for something.")
    
    def test_sample_1(self):
        self.assertEqual(self.sample_1[0][0], 6, "Word count should be 6.")
        
    def test_sample_2(self):
        self.assertEqual(self.sample_2[0][0], 8, "Word count should be 8.")
    
    def test_sample_3(self):
        self.assertEqual(self.sample_3[0][0], 6, "Word count should be 6.")
        
    def test_sample_4(self):
        self.assertEqual(self.sample_4[0][0], 20, "Word count should be 20.")

tests = TestWordCountClass()

tests_loaded = unittest.TestLoader().loadTestsFromModule(tests)

unittest.TextTestRunner().run(tests_loaded);

....
----------------------------------------------------------------------
Ran 4 tests in 0.013s

OK


#### Number of characters
Let's just use the total number of characters.

In [136]:
len('I went to the shops today.')

26

In [11]:
class CharacterCount(BaseEstimator, TransformerMixin):
    def character_count(self, text):
        return len(text)
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        count = pd.Series(x).apply(self.character_count)
        return pd.DataFrame(count)

In [141]:
class TestCharacterCountClass(unittest.TestCase):
    def setUp(self):
        self.sample_1 = CharacterCount().fit_transform('I went to the shops today.')
        self.sample_2 = CharacterCount().fit_transform("They've been very good to me, you know?")
        self.sample_3 = CharacterCount().fit_transform("In they went. Two by two.")
        self.sample_4 = CharacterCount().fit_transform("It takes a lot to love someone. You would think that loving someone for 20 years would count for something.")
        
    def test_sample_1(self):
        self.assertEqual(self.sample_1[0][0], 26, 'Count should be 26.')
    
    def test_sample_2(self):
        self.assertEqual(self.sample_2[0][0], 39, 'Count should be 39.')
    
    def test_sample_3(self):
        self.assertEqual(self.sample_3[0][0], 25, 'Count should be 25.')
    
    def test_sample_4(self):
        self.assertEqual(self.sample_4[0][0], 107, 'Count should be 107.')

tests = TestCharacterCountClass()
tests_loaded = unittest.TestLoader().loadTestsFromModule(tests)
unittest.TextTestRunner().run(tests_loaded);

....
----------------------------------------------------------------------
Ran 4 tests in 0.011s

OK


#### Number of nouns

In [156]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [12]:
class NounCount(BaseEstimator, TransformerMixin):
    def noun_count(self, text):
        count = 0
        sentence_list = sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            for _, tag in pos_tags:
                if tag in ['PRP', 'NN']:
                    count += 1
        
        return count
            
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        count = pd.Series(x).apply(self.noun_count)
        return pd.DataFrame(count)

In [158]:
class TestNounCountClass(unittest.TestCase):
    def setUp(self):
        self.sample_1 = NounCount().fit_transform('I went to the shops today.')
        self.sample_2 = NounCount().fit_transform("They've been very good to me, you know?")
        self.sample_3 = NounCount().fit_transform("In they went; two by two.")
        self.sample_4 = NounCount().fit_transform("It takes a lot to love someone. You would think that loving someone for 20 years would count for something.")
        
    def test_sample_1(self):
        self.assertEqual(self.sample_1[0][0], 2, 'Count should be 2.')
    
    def test_sample_2(self):
        self.assertEqual(self.sample_2[0][0], 0, 'Count should be 0.')
    
    def test_sample_3(self):
        self.assertEqual(self.sample_3[0][0], 0, 'Count should be 0.')
    
    def test_sample_4(self):
        self.assertEqual(self.sample_4[0][0], 5, 'Count should be 5.')

tests = TestNounCountClass()
tests_loaded = unittest.TestLoader().loadTestsFromModule(tests)
unittest.TextTestRunner().run(tests_loaded);

....
----------------------------------------------------------------------
Ran 4 tests in 0.064s

OK


#### Number of verbs

In [21]:
class VerbCount(BaseEstimator, TransformerMixin):
    def verb_count(self, text):
        count = 0
        sentence_list = sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            for word, tag in pos_tags:
                if tag in ['VB', 'VBP']:
                    count += 1
        
        return count
            
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        count = pd.Series(x).apply(self.verb_count)
        return pd.DataFrame(count)

In [20]:
nltk.pos_tag(tokenize('Do you know?'))

[('know', 'VB')]

In [22]:
class TestVerbCountClass(unittest.TestCase):
    def setUp(self):
        self.sample_1 = VerbCount().fit_transform('I went to the shops today.')
        self.sample_2 = VerbCount().fit_transform("They've been very good to me, you know?")
        self.sample_3 = VerbCount().fit_transform("In they went; two by two.")
        self.sample_4 = VerbCount().fit_transform("It takes a lot to love someone. You would think that loving someone for 20 years would count for something.")
        
    def test_sample_1(self):
        self.assertEqual(self.sample_1[0][0], 1, 'Count should be 1.')
    
    def test_sample_2(self):
        self.assertEqual(self.sample_2[0][0], 0, 'Count should be 0.')
        # Should actually be 1 for "know" but not recognized with the pos_tagger rules
    
    def test_sample_3(self):
        self.assertEqual(self.sample_3[0][0], 1, 'Count should be 1.')
    
    def test_sample_4(self):
        self.assertEqual(self.sample_4[0][0], 4, 'Count should be 4.')

tests = TestVerbCountClass()
tests_loaded = unittest.TestLoader().loadTestsFromModule(tests)
unittest.TextTestRunner().run(tests_loaded);

....
----------------------------------------------------------------------
Ran 4 tests in 0.061s

OK


In [52]:
pipeline = Pipeline([
    ("features", FeatureUnion([
        ("text", TfidfVectorizer(tokenizer=tokenize)),
        ("word_count", WordCount()),
        ("character_count", CharacterCount()),
        ("noun_count", NounCount()),
        ("verb_count", VerbCount())
    ])),
    ("clf", MultiOutputClassifier(RandomForestClassifier(random_state=42, verbose=5)))
])

In [42]:
pipeline.fit(train_X, train_Y)
y_preds = pipeline.predict(test_X)

In [43]:
results_model = model_performance(test_Y, y_preds)

related
              precision    recall  f1-score   support

           0       0.57      0.48      0.52      1563
           1       0.85      0.88      0.86      4991

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.71      0.68      0.69      6554
weighted avg       0.78      0.79      0.78      6554

request
              precision    recall  f1-score   support

           0       0.88      0.98      0.93      5443
           1       0.82      0.36      0.50      1111

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.85      0.67      0.72      6554
weighted avg       0.87      0.88      0.86      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      655

C:\Users\rebecca-barnes\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


other_aid
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5690
           1       0.57      0.04      0.07       864

   micro avg       0.87      0.87      0.87      6554
   macro avg       0.72      0.52      0.50      6554
weighted avg       0.83      0.87      0.82      6554

infrastructure_related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6143
           1       1.00      0.00      0.00       411

   micro avg       0.94      0.94      0.94      6554
   macro avg       0.97      0.50      0.49      6554
weighted avg       0.94      0.94      0.91      6554

transport
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6251
           1       0.40      0.01      0.01       303

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.68      0.50      0.49      6554
weighted avg       0.93      0.

That reduced performance, but perhaps with tuning, it's possible to see benefits.

In [53]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text', 'features__word_count', 'features__character_count', 'features__noun_count', 'features__verb_count', 'features__text__analyzer', 'features__text__binary', 'features__text__decode_error', 'features__text__dtype', 'features__text__encoding', 'features__text__input', 'features__text__lowercase', 'features__text__max_df', 'features__text__max_features', 'features__text__min_df', 'features__text__ngram_range', 'features__text__norm', 'features__text__preprocessor', 'features__text__smooth_idf', 'features__text__stop_words', 'features__text__strip_accents', 'features__text__sublinear_tf', 'features__text__token_pattern', 'features__text__tokenizer', 'features__text__use_idf', 'features__text__vocabulary', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_f

In [56]:
make_scorer(f1_score, average='micro')

parameters = {
    'features__text__ngram_range': ((1, 1), (1, 2)),
    'features__text__max_df': [0.5],
    'features__text__max_features': [5000],
    'features__text__use_idf': [False],
    'clf__estimator__max_depth': (25, 50, 100, 200),
    'clf__estimator__min_samples_split': (2, 25, 100), 
    'clf__estimator__n_estimators': (200, 300)
}

cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=3)

In [57]:
cv.fit(train_X, train_Y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 300
building tree 2 of 300
building tree 3 of 300
building tree 4 of 300


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


building tree 5 of 300
building tree 6 of 300
building tree 7 of 300
building tree 8 of 300
building tree 9 of 300
building tree 10 of 300
building tree 11 of 300
building tree 12 of 300
building tree 13 of 300
building tree 14 of 300
building tree 15 of 300
building tree 16 of 300
building tree 17 of 300
building tree 18 of 300
building tree 19 of 300
building tree 20 of 300
building tree 21 of 300
building tree 22 of 300
building tree 23 of 300
building tree 24 of 300
building tree 25 of 300
building tree 26 of 300
building tree 27 of 300
building tree 28 of 300
building tree 29 of 300
building tree 30 of 300
building tree 31 of 300
building tree 32 of 300
building tree 33 of 300
building tree 34 of 300
building tree 35 of 300
building tree 36 of 300
building tree 37 of 300
building tree 38 of 300
building tree 39 of 300
building tree 40 of 300
building tree 41 of 300
building tree 42 of 300
building tree 43 of 300
building tree 44 of 300
building tree 45 of 300
building tree 46 of 3

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   15.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


...

building tree 3 of 300
building tree 4 of 300
building tree 5 of 300
building tree 6 of 300


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


building tree 7 of 300
building tree 8 of 300
building tree 9 of 300
building tree 10 of 300
building tree 11 of 300
building tree 12 of 300
building tree 13 of 300
building tree 14 of 300
building tree 15 of 300
building tree 16 of 300
building tree 17 of 300
building tree 18 of 300
building tree 19 of 300
building tree 20 of 300
building tree 21 of 300
building tree 22 of 300
building tree 23 of 300
building tree 24 of 300
building tree 25 of 300
building tree 26 of 300
building tree 27 of 300
building tree 28 of 300
building tree 29 of 300
building tree 30 of 300
building tree 31 of 300
building tree 32 of 300
building tree 33 of 300
building tree 34 of 300
building tree 35 of 300
building tree 36 of 300
building tree 37 of 300
building tree 38 of 300
building tree 39 of 300
building tree 40 of 300
building tree 41 of 300
building tree 42 of 300
building tree 43 of 300
building tree 44 of 300
building tree 45 of 300
building tree 46 of 300
building tree 47 of 300
building tree 48 of

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   15.3s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngr...           oob_score=False, random_state=42, verbose=5, warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=3,
       param_grid={'features__text__ngram_range': ((1, 1), (1, 2)), 'features__text__max_df': [0.5], 'features__text__max_features': [5000], 'features__text__use_idf': [False], 'clf__estimator__max_depth': (25, 50, 100, 200), 'clf__estimator__min_samples_split': (2, 25, 100), 'clf__estimator__n_estimators': (200, 300)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [58]:
cv.best_params_

{'clf__estimator__max_depth': 200,
 'clf__estimator__min_samples_split': 25,
 'clf__estimator__n_estimators': 300,
 'features__text__max_df': 0.5,
 'features__text__max_features': 5000,
 'features__text__ngram_range': (1, 2),
 'features__text__use_idf': False}

In [59]:
cv.best_score_

0.26960634726884347

In [60]:
y_preds = cv.predict(test_X)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.7s finished


In [62]:
cv_results = model_performance(test_Y, y_preds)

related
              precision    recall  f1-score   support

           0       0.73      0.31      0.43      1563
           1       0.82      0.96      0.88      4991

   micro avg       0.81      0.81      0.81      6554
   macro avg       0.77      0.64      0.66      6554
weighted avg       0.79      0.81      0.78      6554

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5443
           1       0.85      0.47      0.60      1111

   micro avg       0.90      0.90      0.90      6554
   macro avg       0.88      0.72      0.77      6554
weighted avg       0.89      0.90      0.88      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      655

C:\Users\rebecca-barnes\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6339
           1       0.50      0.08      0.14       215

   micro avg       0.97      0.97      0.97      6554
   macro avg       0.73      0.54      0.56      6554
weighted avg       0.95      0.97      0.96      6554

death
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6257
           1       0.85      0.32      0.46       297

   micro avg       0.97      0.97      0.97      6554
   macro avg       0.91      0.66      0.72      6554
weighted avg       0.96      0.97      0.96      6554

other_aid
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5690
           1       0.70      0.04      0.08       864

   micro avg       0.87      0.87      0.87      6554
   macro avg       0.79      0.52      0.50      6554
weighted avg       0.85      0.87      0.82      6554

inf

This is an improvement over the best model so far. The fact that the best model is associated with the highest available ranges for max depth and number of estimators suggests that the model may be improved by increasing these values.

In [64]:
pipeline = Pipeline([
    ("features", FeatureUnion([
        ("text", TfidfVectorizer(tokenizer=tokenize, max_df=0.5, 
                                 max_features=5000, ngram_range=(1, 2),
                                 use_idf=False)),
        ("word_count", WordCount()),
        ("character_count", CharacterCount()),
        ("noun_count", NounCount()),
        ("verb_count", VerbCount())
    ])),
    ("clf", MultiOutputClassifier(RandomForestClassifier(min_samples_split=25, random_state=42, verbose=3)))
])

In [65]:
scorer = make_scorer(f1_score, average='micro')

parameters = {
    'clf__estimator__max_depth': (200, 350, 500),
    'clf__estimator__n_estimators': (300, 450, 600)
}

cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=3, scoring=scorer)

In [66]:
cv.fit(train_X, train_Y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 300
building tree 2 of 300
building tree 3 of 300
building tree 4 of 300
building tree 5 of 300
building tree 6 of 300
building tree 7 of 300
building tree 8 of 300
building tree 9 of 300
building tree 10 of 300
building tree 11 of 300
building tree 12 of 300
building tree 13 of 300
building tree 14 of 300
building tree 15 of 300
building tree 16 of 300
building tree 17 of 300
building tree 18 of 300
building tree 19 of 300
building tree 20 of 300
building tree 21 of 300
building tree 22 of 300
building tree 23 of 300
building tree 24 of 300
building tree 25 of 300
building tree 26 of 300
building tree 27 of 300
building tree 28 of 300
building tree 29 of 300
building tree 30 of 300
building tree 31 of 300
building tree 32 of 300
building tree 33 of 300
building tree 34 of 300
building tree 35 of 300
building tree 36 of 300
building tree 37 of 300
building tree 38 of 300
building tree 39 of 300
building tree 40 of 300
building tree 41 of 300
building tree 42 of 300
b

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   23.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 300
building tree 2 of 300
building tree 3 of 300


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


...

building tree 1 of 300
building tree 2 of 300
building tree 3 of 300


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


building tree 4 of 300
building tree 5 of 300
building tree 6 of 300
building tree 7 of 300
building tree 8 of 300
building tree 9 of 300
building tree 10 of 300
building tree 11 of 300
building tree 12 of 300
building tree 13 of 300
building tree 14 of 300
building tree 15 of 300
building tree 16 of 300
building tree 17 of 300
building tree 18 of 300
building tree 19 of 300
building tree 20 of 300
building tree 21 of 300
building tree 22 of 300
building tree 23 of 300
building tree 24 of 300
building tree 25 of 300
building tree 26 of 300
building tree 27 of 300
building tree 28 of 300
building tree 29 of 300
building tree 30 of 300
building tree 31 of 300
building tree 32 of 300
building tree 33 of 300
building tree 34 of 300
building tree 35 of 300
building tree 36 of 300
building tree 37 of 300
building tree 38 of 300
building tree 39 of 300
building tree 40 of 300
building tree 41 of 300
building tree 42 of 300
building tree 43 of 300
building tree 44 of 300
building tree 45 of 30

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   21.9s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=5000, min_df=1,
        ngr...           oob_score=False, random_state=42, verbose=3, warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=3,
       param_grid={'clf__estimator__max_depth': (200, 350, 500), 'clf__estimator__n_estimators': (300, 450, 600)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(f1_score, average=micro), verbose=0)

In [67]:
cv.best_score_

0.667859308775882

In [68]:
cv.best_params_

{'clf__estimator__max_depth': 500, 'clf__estimator__n_estimators': 300}

In [69]:
y_preds = cv.predict(test_X)
results_cv = model_performance(test_Y, y_preds)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concur

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    1.0s finish

related
              precision    recall  f1-score   support

           0       0.72      0.33      0.45      1563
           1       0.82      0.96      0.89      4991

   micro avg       0.81      0.81      0.81      6554
   macro avg       0.77      0.64      0.67      6554
weighted avg       0.80      0.81      0.78      6554

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5443
           1       0.85      0.47      0.61      1111

   micro avg       0.90      0.90      0.90      6554
   macro avg       0.88      0.73      0.77      6554
weighted avg       0.89      0.90      0.88      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      655

              precision    recall  f1-score   support

           0       0.91      0.94      0.92      4781
           1       0.83      0.74      0.78      1773

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.87      0.84      0.85      6554
weighted avg       0.88      0.89      0.89      6554

floods
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      6035
           1       0.87      0.55      0.67       519

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.91      0.77      0.83      6554
weighted avg       0.95      0.96      0.95      6554

storm
              precision    recall  f1-score   support

           0       0.97      0.98      0.97      5949
           1       0.74      0.66      0.70       605

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.86      0.82      0.84      6554
weighted avg       0.95      0.95      0.95      6554

fire
 

This appears to be the best model so far, including having predictions for all classes. 

### 9. Export your model as a pickle file

In [72]:
with open('data/model.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.